In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
tf.__version__

'2.10.0'

In [4]:
# 3.5 新聞分類：多分類問題
# §代碼清單 3-12

from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)

In [5]:
# === FOR CHECK ===
print('train records : ', len(train_data))
print('test  records : ', len(test_data))
print('\n')
print('train_data[10] :')
print(train_data[10])

train records :  8982
test  records :  2246


train_data[10] :
[1, 245, 273, 207, 156, 53, 74, 160, 26, 14, 46, 296, 26, 39, 74, 2979, 3554, 14, 46, 4689, 4329, 86, 61, 3499, 4795, 14, 61, 451, 4329, 17, 12]


In [6]:
# §代碼清單 3-13 將索引解碼為新聞文本

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[10]])

print(decoded_newswire)

? period ended december 31 shr profit 11 cts vs loss 24 cts net profit 224 271 vs loss 511 349 revs 7 258 688 vs 7 200 349 reuter 3


In [7]:
print(train_labels[10])

3


In [8]:
# §代碼清單 3-14 數據編碼

import numpy as np

def vectorize_sequences(sequences, dimension = 10000) :
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences) :
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [9]:
# §代碼清單 3-14 數據編碼 continue ...

def to_one_hot(labels, dimension = 46) :
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels) :
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

# built-in metho in keras
#from keras.utils.np_utils import to_categorical
#one_hot_train_labels = to_categorical(train_labels)
#one_hot_test_labels = to_categorical(test_labels)



In [10]:
# §代碼清單 3-15 模型定義

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))


In [11]:
# §代碼清單 3-16 編譯模型

model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


In [12]:
# §代碼清單 3-17 留出驗證集

x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [13]:
# §代碼清單 3-18 訓練模型

history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs = 20,
                   batch_size = 512,
                   validation_data = (x_val, y_val))


Epoch 1/20
16/16 [==============================] - 4s 91ms/step - loss: 2.5470 - accuracy: 0.5464 - val_loss: 1.6698 - val_accuracy: 0.6490
Epoch 2/20
16/16 [==============================] - 1s 51ms/step - loss: 1.3723 - accuracy: 0.7081 - val_loss: 1.2724 - val_accuracy: 0.7150
Epoch 3/20
16/16 [==============================] - 1s 41ms/step - loss: 1.0354 - accuracy: 0.7749 - val_loss: 1.1200 - val_accuracy: 0.7570
Epoch 4/20
16/16 [==============================] - 1s 46ms/step - loss: 0.8215 - accuracy: 0.8259 - val_loss: 1.0188 - val_accuracy: 0.7860
Epoch 5/20
16/16 [==============================] - 1s 42ms/step - loss: 0.6596 - accuracy: 0.8564 - val_loss: 0.9526 - val_accuracy: 0.7960
Epoch 6/20
16/16 [==============================] - 1s 45ms/step - loss: 0.5285 - accuracy: 0.8870 - val_loss: 0.9668 - val_accuracy: 0.7830
Epoch 7/20
16/16 [==============================] - 1s 46ms/step - loss: 0.4264 - accuracy: 0.9107 - val_loss: 0.9060 - val_accuracy: 0.7980
Epoch 8/20
16

In [ ]:
# §代碼清單 3-19 繪製訓練損失和驗證損失

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# §代碼清單 3-20 繪製訓練精度和驗證精度

plt.clf()


In [ ]:
# §代碼清單 3-21 從頭開始重新訓練一個模型

model = models.Sequential()
